# अध्याय 2 भाषा मॉडल, प्रश्नोत्तरी प्रतिमान और टोकन

- [आई. पर्यावरण विन्यास](#आई. पर्यावरण विन्यास)
- [1.1 एपीआई कुंजी और कुछ पायथन लाइब्रेरीज़ लोड हो रही है।)
- [1.2 हेल्पर फ़ंक्शन सहायक फ़ंक्शन](#1.2-हेल्पर-फ़ंक्शन-सहायक फ़ंक्शन)
- [II. मॉडल से प्रश्न पूछने का प्रयास करें और परिणाम प्राप्त करें](#II. मॉडल से प्रश्न पूछने का प्रयास करें और परिणाम प्राप्त करें)
- [III. टोकन](#III. टोकन)
- [IV. सहायक कार्य सहायक कार्य (प्रश्न)tioning प्रतिमान)](#IV। सहायक-कार्य-सहायक कार्य-(प्रश्न प्रतिमान))

इस अध्याय में, हम आपके साथ साझा करेंगे कि बड़े भाषा मॉडल (एलएलएम) कैसे काम करते हैं, उन्हें कैसे प्रशिक्षित किया जाता है, और टोकनाइज़र जैसे विवरण एलएलएम आउटपुट को कैसे प्रभावित करते हैं। हम एलएलएम के चैट प्रारूप, सिस्टम संदेशों और उपयोगकर्ता संदेशों को निर्दिष्ट करने का एक तरीका भी पेश करेंगे , और आपको दिखाएंगे कि इस क्षमता का लाभ कैसे उठाया जाए।

## 1. पर्यावरण विन्यास

### 1.1 एपीआई कुंजी और कुछ पायथन लाइब्रेरी लोड हो रही है।
इस कोर्स में, आपको OpenAI API कुंजी लोड करने के लिए कुछ कोड प्रदान किए जाते हैं।

In [ ]:
!pip install openai
!pip install langchain
!pip install --upgrade tiktoken

In [1]:
import os
import openai
# आयात टिकटोकन इसका बाद में उपयोग नहीं किया जाएगा। यदि आप इसके उपयोग में रुचि रखते हैं, तो आप संबंधित सामग्री के लिए इस लेख को देख सकते हैं: https://zhuanlan.zhihu.com/p/629776230

# dotenv से आयात करेंload_dotenv, ढूंढें_dotenv
# _ =load_dotenv(find_dotenv()) # स्थानीय .env पर्यावरण फ़ाइल पढ़ें (भविष्य में इस पद्धति का उपयोग करने और अपनी कुंजी की सुरक्षा के लिए कुंजी को .env फ़ाइल में डालने की अनुशंसा की जाती है)

openai.api_key  = 'sk-***' # 更换成您自己的key

### 1.2 सहायक कार्य
यदि आपने पहले "चैटजीपीटी प्रॉम्प्ट इंजीनियरिंग फॉर डेवलपर्स" पाठ्यक्रम लिया है, तो आप इससे अपेक्षाकृत परिचित होंगे।
इस फ़ंक्शन को कॉल करें और प्रॉम्प्ट दर्ज करें, और यह संबंधित पूर्णता देगा।

In [2]:
# आधिकारिक दस्तावेज़ लेखन https://platform.openai.com/overview

def get_completion(prompt, model="gpt-3.5-turbo"):
    """
    使用 OpenAI 的模型生成聊天回复。

    参数:
    prompt: 用户的输入，即聊天的提示。
    model: 使用的模型，默认为"gpt-3.5-turbo"。
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"] # 模型生成的回复

## 2. मॉडल प्रश्न पूछने और परिणाम प्राप्त करने का प्रयास करें

एलएलएम पर्यवेक्षित शिक्षण का उपयोग करके बनाया जा सकता है, जहां वे बार-बार अगले शब्द की भविष्यवाणी करके सीखते हैं और, एक बड़े प्रशिक्षण सेट, दसियों अरबों या अधिक शब्दों को देखते हुए, आप एक बड़े पैमाने पर प्रशिक्षण सेट बना सकते हैं जहां आप एक वाक्य से शुरू करते हैं। किसी पाठ का भाग और बार-बार भाषा मॉडल से यह अनुमान लगाना सीखें कि अगला शब्द क्या है।

एलएलएम के दो मुख्य प्रकार हैं: बेस एलएलएम और इंस्ट्रक्शन ट्यून्ड एलएलएम, जो तेजी से लोकप्रिय हो रहे हैं। बेस एलएलएम को बार-बार अगले शब्द की भविष्यवाणी करके प्रशिक्षित किया जाता है, इसलिए यदि हम इसे एक संकेत देते हैं जैसे "एक बार एक गेंडा था,"। "यह शब्द दर शब्द भविष्यवाणी करके एक जादुई जंगल में अपने अन्य गेंडा दोस्तों के साथ गेंडा के जीवन के बारे में एक कहानी को पूरा करने में सक्षम हो सकता है।

हालाँकि, इस दृष्टिकोण का नकारात्मक पक्ष यह है कि यदि आप इसे "चीन की राजधानी क्या है?" जैसा संकेत देते हैं, तो यह'संभावना है कि इसके डेटा में इंटरनेट से चीन के बारे में प्रश्नोत्तरी प्रश्नों की एक सूची है, इस बिंदु पर, यह इस प्रश्न का उत्तर दे सकता है कि "चीन का सबसे बड़ा शहर कौन सा है? चीन की जनसंख्या कितनी है?" लेकिन वास्तव में, आप केवल यह जानना चाहते हैं कि चीन की राजधानी क्या है, इन सभी प्रश्नों को सूचीबद्ध नहीं करना है, हालांकि, एक इंस्ट्रक्शन ट्यून्ड एलएलएम संकेत का पालन करने और उत्तर देने का प्रयास करेगा "चीन की राजधानी बीजिंग है।"

तो आप बेस एलएलएम को इंस्ट्रक्शन ट्यून्ड एलएलएम में कैसे बदल सकते हैं?यह एक इंस्ट्रक्शन ट्यून्ड एलएलएम (जैसे चैटजीपीटी) को प्रशिक्षित करने की प्रक्रिया है, सबसे पहले, आपको बेस एलएलएम को बहुत सारे डेटा पर प्रशिक्षित करने की आवश्यकता है, इसलिए आपको सैकड़ों अरब शब्दों की आवश्यकता है, या इससे भी अधिक बड़े सुपरकंप्यूटिंग सिस्टम। बेस भाषा मॉडल को प्रशिक्षित करने के बाद, आप इसे मॉडल के आउटपुट को इनपुट से मेल खाने के लिए उदाहरणों के एक छोटे सेट पर प्रशिक्षित करते हैं। उदाहरण के लिए, आप एक ठेकेदार से निर्देशों के कई उदाहरण लिखने और मॉडल को प्रशिक्षित करने में मदद करने के लिए कह सकते हैं कोर परयह उन निर्देशों के प्रति प्रतिक्रियाएँ जोड़ता है। यह फ़ाइन-ट्यूनिंग के लिए एक प्रशिक्षण सेट बनाता है, जहाँ मॉडल निर्देशों का पालन करते समय अगले शब्द की भविष्यवाणी करना सीखता है।

बाद में, भाषा मॉडल आउटपुट की गुणवत्ता में सुधार करने के लिए, एक सामान्य दृष्टिकोण यह है कि मनुष्य कई अलग-अलग आउटपुट को रेट करें, जैसे कि उपयोगी, सच्चा, हानिरहित, आदि। फिर आप उच्च रेटिंग उत्पन्न करने की संभावना बढ़ाने के लिए भाषा मॉडल को और अधिक ट्यून कर सकते हैं। आउटपुट। यह अक्सर सुदृढीकरण सीखने वाली मानव प्रतिक्रिया (आरएलएचएफ) का उपयोग करके किया जाता है।तकनीकें। एक आधार भाषा मॉडल को प्रशिक्षित करने में लगने वाले महीनों की तुलना में, छोटे डेटासेट और कंप्यूटिंग संसाधनों का उपयोग करके, एक आधार भाषा मॉडल से एक निर्देश-सुसंस्कृत भाषा मॉडल में संक्रमण में केवल कुछ दिन लग सकते हैं।

In [13]:
response = get_completion("What is the capital of China?")
print(response)

The capital of China is Beijing.


In [14]:
response = get_completion("中国的首都是哪里？")
print(response)

中国的首都是北京。


##टोकन

एलएलएम के अब तक के विवरण में, हमने इसे एक समय में एक शब्द की भविष्यवाणी के रूप में वर्णित किया है, लेकिन वास्तव में एक अधिक महत्वपूर्ण तकनीकी विवरण है, **`एलएलएम वास्तव में अगले शब्द की बार-बार भविष्यवाणी नहीं करता है, बल्कि बार-बार भविष्यवाणी करता है अगला टोकन`**। जब एलएलएम इनपुट प्राप्त करता है, तो यह इसे टोकन की एक श्रृंखला में परिवर्तित करता है, जहां प्रत्येक टोकन वर्णों के एक सामान्य अनुक्रम का प्रतिनिधित्व करता है, उदाहरण के लिए, वाक्य के लिए "नई चीजें सीखना मजेदार है!", प्रत्येक शब्द में परिवर्तित हो जाता है एक टोकन, और कम आम के लिएजैसे शब्द "प्रॉम्पटिंग को शक्तिशाली डेवलपर टूल के रूप में", "प्रॉम्पटिंग" शब्द को तीन टोकन में विभाजित किया गया है, अर्थात् "प्रोम", "पीटी", और "आईएनजी"।

जब आप ChatGPT को "लॉलीपॉप" के अक्षरों को उलटने के लिए कहते हैं, तो ChatGPT को अक्षरों का सही क्रम आउटपुट करने में कठिनाई होती है क्योंकि टोकनाइज़र "लॉलीपॉप" को तीन टोकन में तोड़ देता है, अर्थात् "l", "oll", "ipop"। चैटजीपीटी किसी शब्द के प्रत्येक अक्षर को बेहतर ढंग से पहचानता है और अक्षरों के बीच हाइफ़न या रिक्त स्थान जोड़कर उन्हें सही ढंग से आउटपुट करता है ताकि टोकननाइज़रप्रत्येक अक्षर को अलग-अलग टोकन में तोड़ता है..

In [15]:
# प्रभाव को बेहतर ढंग से प्रदर्शित करने के लिए, यहां कोई संकेत चीनी भाषा में अनुवादित नहीं किया गया है
# ध्यान दें कि यहां अक्षर फ़्लिप करना गलत है। एंड्रयू एनजी इस उदाहरण का उपयोग यह समझाने के लिए करते हैं कि टोकन की गणना कैसे की जाती है।
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


"लॉलीपॉप" का उल्टा "पोपिलोल" होना चाहिए

In [17]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

print(response)

p-o-p-i-l-l-o-l


![Tokens.png](../../figures/Tokens.png)

अंग्रेजी इनपुट के लिए, एक टोकन आम तौर पर 4 अक्षरों या एक शब्द के तीन चौथाई से मेल खाता है; चीनी इनपुट के लिए, एक टोकन आम तौर पर एक या आधे शब्द से मेल खाता है।

विभिन्न मॉडलों में अलग-अलग टोकन सीमाएं होती हैं। यह ध्यान दिया जाना चाहिए कि यहां टोकन सीमा इनपुट प्रॉम्प्ट और आउटपुट पूर्णता टोकन का योग है। इसलिए, इनपुट प्रॉम्प्ट जितना लंबा होगा, आउटपुट की पूर्णता की ऊपरी सीमा उतनी ही कम होगी।

ChatGPT3.5-टर्बो की टोकन ऊपरी सीमा 4096 है।

## 4. सहायक कार्य (प्रश्न प्रतिमान)
पाठ्यक्रम में प्रयुक्त सहायक कार्य निम्नलिखित हैं।
निम्नलिखित आंकड़ा ओपनएआई द्वारा प्रदान किया गया एक प्रतिमान है। इसके बाद, श्री एंड्रयू एनजी दर्शाते हैं कि बेहतर प्रश्न पूछने के लिए इस प्रतिमान का उपयोग कैसे किया जाए
![Chat-format.png](../../figures/Chat-format.png)

सिस्टम जानकारी का उपयोग मॉडल के नियमों को निर्दिष्ट करने के लिए किया जाता है, जैसे सेटिंग्स और उत्तर मानदंड, जबकि सहायक जानकारी मॉडल को पूरा करने के लिए विशिष्ट निर्देश है।

In [5]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

In [6]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, a carrot would sprout,
With a cheery orange hue, without a doubt.
With a leafy green top, it danced in the breeze,
A happy carrot, so eager to please.

It grew in the soil, oh so deep and grand,
Stretching its roots, reaching far and expand.
With a joyful smile, it soaked up the sun,
Growing tall and strong, its journey begun.

Days turned to weeks, as it grew day and night,
Round and plump, it was quite a delight.
With every raindrop that fell from above,
The carrot grew sweeter, spreading more love.

At last, the day came when it was time to eat,
With a grin on my face, I took a seat.
I chopped and I sliced, so grateful, you see,
For this happy carrot, bringing joy to me.

So let us remember, when times may get tough,
A happy carrot's journey, it's enough.
For even in darkness, there's always delight,
Just like a carrot, shining so bright.


In [18]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

在海洋的深处，有一只小鲸鱼，
她快乐又聪明，灵感从不匮乏。
她游遍五大洲，探索未知的秘密，
用歌声传递喜悦，令人心旷神怡。

她跃出海面，高高的飞翔，
尾巴抽空着水花，像梦幻般的画。
她和海豚一起，跳跃在太阳下，
与海洋中的生命，在欢乐中共舞。

她喜欢和海龟一起，缓缓漫游，
看美丽的珊瑚，和色彩鲜艳的鱼群。
她欢迎每个新朋友，无论大或小，
因为在她眼中，每个人都独特而珍贵。

她知道快乐是如此简单，如此宝贵，
在每个时刻中，她都努力传达幸福的表情。
所以当你感到疲惫，沮丧或者低落，
想起小鲸鱼的快乐，让你心中再次充满鲜活。


In [7]:
# लंबाई
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a cheerful carrot named Charlie who always brightened everyone's day with his vibrant orange color and contagious laughter.


In [19]:
#लंबाई नियंत्रण
messages =  [  
{'role':'system',
 'content':'你的所有答复只能是一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在追随波浪的起伏中，小鲸鱼快乐地跳跃着，因为它知道游泳的真正乐趣不仅仅在目的地，而是在于享受整个旅程。


In [8]:
#संयुक्त
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once upon a time, there was a carrot so happy and bright, it danced and sang from morning till night.


In [20]:
# उपरोक्त का संयोजन
messages =  [  
{'role':'system',
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答，只回答一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在蓝色的大海里，有一只小鲸鱼，无忧无虑，快乐游泳，一切因快乐而变得光辉。


In [9]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    """
    使用 OpenAI 的 GPT-3 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。

    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-3.5-turbo"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。

    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [24]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

In a garden so bright, with colors so cheery,
There lived a carrot, oh so merry!
With a vibrant orange hue, and a leafy green top,
This happy carrot just couldn't stop.

It danced in the breeze, with a joyful sway,
Spreading happiness throughout the day.
With a smile so wide, and eyes full of glee,
This carrot was as happy as can be.

It loved the sunshine, and the rain's gentle touch,
Growing tall and strong, oh so much!
From the earth it sprouted, reaching for the sky,
A happy carrot, oh my, oh my!

So if you're feeling down, just remember this tale,
Of a carrot so happy, it'll never fail.
Find joy in the little things, and let your heart sing,
Just like that carrot, oh what joy it will bring!


In [22]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

In [25]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 173, 'total_tokens': 210}


अंत में, हमारा मानना ​​​​है कि एआई एप्लिकेशन विकास पर प्रॉम्प्ट के क्रांतिकारी प्रभाव को अभी भी कम सराहा गया है, पारंपरिक पर्यवेक्षित मशीन लर्निंग वर्कफ़्लो में, यदि आप एक क्लासिफायरियर बनाना चाहते हैं जो रेस्तरां समीक्षाओं को सकारात्मक या नकारात्मक के रूप में वर्गीकृत कर सकता है, तो आपको पहले एक बड़ा प्राप्त करना होगा। लेबल किए गए डेटा की मात्रा, शायद उनमें से सैकड़ों, और इस प्रक्रिया में सप्ताह या एक महीना भी लग सकता है, फिर आपको इस डेटा पर एक मॉडल को प्रशिक्षित करने, एक उपयुक्त ओपन सोर्स मॉडल ढूंढने और मॉडल ट्यूनिंग करने की आवश्यकता हैमूल्यांकन, जिसमें दिन, सप्ताह या महीने भी लग सकते हैं, अंत में, आपको मॉडल को तैनात करने, मॉडल को क्लाउड पर अपलोड करने और अंततः अपने मॉडल को कॉल करने से पहले इसे चलाने की आवश्यकता हो सकती है इसे पूरा करने में एक टीम को कई महीने लग जाते हैं।

इसके विपरीत, प्रॉम्प्ट-आधारित मशीन लर्निंग दृष्टिकोण के साथ, जब आपके पास एक टेक्स्ट एप्लिकेशन होता है, तो आप बस एक साधारण प्रॉम्प्ट प्रदान करते हैं। इस प्रक्रिया में केवल कुछ मिनट या अधिकतम कुछ घंटे लग सकते हैं यदि इसमें कई पुनरावृत्तियाँ लगती हैंएक वैध संकेत। कुछ दिनों के भीतर (हालांकि व्यवहार में यह आमतौर पर कुछ घंटों का होता है), आप एपीआई कॉल के माध्यम से मॉडल चला सकते हैं और इसका उपयोग शुरू कर सकते हैं। एक बार जब आप इस चरण पर पहुंच जाते हैं, तो कॉल शुरू करने में केवल कुछ मिनट या घंटे लगते हैं अनुमान के लिए मॉडल। परिणामस्वरूप, जिन अनुप्रयोगों को बनाने में पहले छह महीने या यहां तक ​​कि एक वर्ष लगता था, उन्हें अब मिनटों, घंटों या अधिकतम दिनों में बनाया जा सकता है, आप इसे प्रॉम्प्ट का उपयोग करके बना सकते हैं एआई अनुप्रयोगों के निर्माण का तरीका बदल रहा हैजल्दी करो.

यह ध्यान रखना महत्वपूर्ण है कि यह दृष्टिकोण कई असंरचित डेटा अनुप्रयोगों, विशेष रूप से पाठ अनुप्रयोगों और तेजी से बढ़ते दृष्टि अनुप्रयोगों के लिए काम करता है, हालांकि वर्तमान दृष्टि तकनीक अभी भी विकसित हो रही है, यह संरचित डेटा अनुप्रयोगों, यानी मशीन लर्निंग अनुप्रयोगों के लिए काम नहीं करती है जो बड़ी प्रक्रिया करते हैं एक्सेल स्प्रेडशीट में मूल्यों की संख्या हालांकि, इस दृष्टिकोण के लिए उपयुक्त अनुप्रयोगों के लिए, एआई घटकों को जल्दी से बनाया जा सकता है और सह को बदल रहे हैंसंपूर्ण सिस्टम के निर्माण कार्यप्रवाह में अभी भी दिन, सप्ताह या अधिक समय लग सकता है, लेकिन कम से कम यह भाग तेजी से किया जा सकता है।

अगले अध्याय में, हम दिखाएंगे कि ग्राहक सेवा सहायक के इनपुट का मूल्यांकन करने के लिए इन घटकों का उपयोग कैसे करें।
यह एक ऑनलाइन रिटेलर के लिए ग्राहक सेवा सहायक के निर्माण के इस पाठ्यक्रम में एक अधिक संपूर्ण उदाहरण का हिस्सा होगा।